In [1]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tltk
tltk.pos_load()
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
%matplotlib inline

In [2]:
# Path to the preprocessed data
best_processed_path = 'w_review_train.csv'

In [3]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'ซ': 104, ',': 13, '๔': 169, 'G': 40, "'": 8, '?': 32, '๐': 165, 'q': 82, '\ufeff': 177, 'k': 75, 'i': 73, 'ื': 147, '๘': 173, 'ั': 141, '8': 25, 'ว': 131, 'D': 37, 'ๅ': 156, '้': 160, 'ศ': 132, 'Q': 50, 'n': 78, 'ฃ': 96, 'ฉ': 102, '๓': 168, ')': 10, '๒': 167, 'S': 52, 'แ': 152, 'ก': 94, 'ุ': 148, 'ง': 100, ':': 27, '.': 15, ';': 28, 'ช': 103, '’': 176, 'ำ': 143, '\n': 0, 'ึ': 146, 'K': 44, '4': 21, '(': 9, 'O': 48, 'ข': 95, '๖': 171, '่': 159, 'N': 47, '๊': 161, '<': 29, 'b': 66, ']': 62, '๋': 162, 't': 85, 'า': 142, 'X': 57, '๑': 166, 'ฏ': 108, 'c': 67, 'a': 65, 'ฅ': 98, 'u': 86, 'ท': 116, 'ษ': 133, 'อ': 137, '๕': 170, 'ร': 128, 'ฑ': 110, '7': 24, 'ฤ': 129, 'ไ': 155, 'ฐ': 109, 'ถ': 115, 'j': 74, 'p': 81, 'ส': 134, 'ด': 113, 'T': 53, '}': 92, 'ู': 149, 'd': 68, 'ฬ': 136, '3': 20, 'ฺ': 150, 'ธ': 117, 'z': 91, 'r': 83, 'ฆ': 99, 'ะ': 140, 'ภ': 125, 'ฮ': 138, 'f': 70, 'E': 38, 'ฝ': 122, 'B': 35, '&': 7, '[': 60, '๙': 174, 'ห': 135, 'l': 76, '@': 33, 'M': 46, 'ผ': 121, 'ฒ': 111, 'v': 87, 

In [4]:
from keras.models import Model
from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
from keras.layers import Input, GRU, Bidirectional
from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)ฅ
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model
    

Using TensorFlow backend.


In [7]:
weight_path_my_best_model2='/data/model_best.h5'
my_best_model2 = get_my_best_model()
my_best_model2.load_weights(weight_path_my_best_model2)
#my_best_model2.make_predict_function()
my_best_model2.summary()

OSError: Unable to open file (unable to open file: name = '/data/model_best.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [8]:
df = pd.read_csv(best_processed_path, sep=';', header=None)

In [ ]:
# df

In [9]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [10]:
word_test = [x for x in df[0][1]]
print(''.join(word_test))

อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก "ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 
อาหารที่เราแนะนำคือไก่เบตง (คล้ายๆไก่แช่เหล้า) เสริฟพร้อมกับหอมเจียว และน้ำจิ้มน้ำพริกเผาสูตรเด็ดของทางร้าน 
เมนูข้าวผัดหนำเลียบก็อร่อยค่ะ ชอบมากๆ


In [11]:
word_test_df2 = pd.DataFrame(word_test)
#print(word_test_df2)

n_pad = 21
n_pad_2 = int((n_pad - 1)/2)
pad = [{0: ' '}]
df_pad = pd.DataFrame(pad * n_pad_2)
#print(df_pad)

word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
#print(word_test_df2[0][:21])
word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))

In [12]:
df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

# convert pandas dataframe to numpy array to feed to the model
x_char = df_with_context[char_row].as_matrix()

In [13]:
y_pred = my_best_model2.predict(x_char)

prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

In [14]:
print(len(y_pred))
print(len(word_test))
word_token = []
word_tmp = ''
for i in range(len(y_pred)):
    if(y_pred[i] == 1):
        word_token.append(word_tmp)
        word_tmp = ''
    word_tmp += word_test[i]
else:
    word_token.append(word_tmp)
word_token = word_token[1:]
print(word_token)

701
701
['อ', 'า', 'ห', 'า', 'ร', 'ท', 'ี', '่', 'น', 'ี', '่', 'เ', 'ป', '็', 'น', 'อ', 'า', 'ห', 'า', 'ร', 'จ', 'ี', 'นแ', 'ค', 'ะ', 'ท', 'ี', '่', 'ห', 'า', 'ก', 'ิ', 'น', 'ย', 'า', 'ก', 'ใ', 'น', 'บ', '้', 'า', 'น', 'เ', 'ร', 'า', ' ', 'ต', 'ั', 'ว', 'ร', '้', 'า', 'น', 'ต', 'ั', '้', 'ง', 'อ', 'ย', 'ู', '่', 'ท', 'ี', '่', 'ถ', 'น', 'น', 'พ', 'ุ', 'ท', 'ธมณ', 'ฑ', 'ล', 'ส', 'า', 'ย', ' ', '3', ' ', 'ไ', 'ป', 'ต', 'า', 'ม', ' ', 'ถ', '.', 'ป', 'ิ', '่', 'น', 'เ', 'ก', 'ล', '้', 'า', '-', 'น', 'ค', 'ร', 'ช', 'ั', 'ย', 'ศ', 'ร', 'ี', ' ', 'เ', 'ม', 'ื', '่', 'อ', 'ถ', 'ึง', 'พ', 'ุ', 'ท', 'ธ', 'มณ', 'ฑ', 'ล', 'ส', 'า', 'ย', ' ', '3', ' ', 'ก', '็', 'เ', 'ล', 'ี', '้', 'ย', 'ว', 'เ', 'ข', '้', 'า', 'ไ', 'ป', 'ป', 'ร', 'ะ', 'ม', 'า', 'ณ', ' ', '50', '0', ' ', 'เ', 'ม', 'ต', 'ร', ' ร', '้', 'า', 'น', 'อ', 'ย', 'ู', '่', 'ท', 'า', 'งซ', '้', 'า', 'ย', 'ม', 'ื', 'อ', 'ค', '่', 'ะ', ' ', 'ม', 'ี', 'ค', 'นบ', 'อ', 'กม', 'า', 'ว', '่', 'า', 'ค', 'ว', 'า', 'ม', 'พ', 'ิ', 'เ', 'ศ', 'ษ', 'ข', '

In [15]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    return word_token

In [16]:
def mergeLabel(review, rating):
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))
    label = [int(rating)] * len(cleaned)
    training_data = list(zip(cleaned, label))
    return training_data

In [17]:
mergeLabel(df[0][1], df[1][1])

[('อ', 4),
 ('า', 4),
 ('ห', 4),
 ('า', 4),
 ('ร', 4),
 ('ท', 4),
 ('ี', 4),
 ('่', 4),
 ('น', 4),
 ('ี', 4),
 ('่', 4),
 ('เ', 4),
 ('ป', 4),
 ('็', 4),
 ('น', 4),
 ('อ', 4),
 ('า', 4),
 ('ห', 4),
 ('า', 4),
 ('ร', 4),
 ('จ', 4),
 ('ี', 4),
 ('นแ', 4),
 ('ค', 4),
 ('ะ', 4),
 ('ท', 4),
 ('ี', 4),
 ('่', 4),
 ('ห', 4),
 ('า', 4),
 ('ก', 4),
 ('ิ', 4),
 ('น', 4),
 ('ย', 4),
 ('า', 4),
 ('ก', 4),
 ('ใ', 4),
 ('น', 4),
 ('บ', 4),
 ('้', 4),
 ('า', 4),
 ('น', 4),
 ('เ', 4),
 ('ร', 4),
 ('า', 4),
 ('ต', 4),
 ('ั', 4),
 ('ว', 4),
 ('ร', 4),
 ('้', 4),
 ('า', 4),
 ('น', 4),
 ('ต', 4),
 ('ั', 4),
 ('้', 4),
 ('ง', 4),
 ('อ', 4),
 ('ย', 4),
 ('ู', 4),
 ('่', 4),
 ('ท', 4),
 ('ี', 4),
 ('่', 4),
 ('ถ', 4),
 ('น', 4),
 ('น', 4),
 ('พ', 4),
 ('ุ', 4),
 ('ท', 4),
 ('ธมณ', 4),
 ('ฑ', 4),
 ('ล', 4),
 ('ส', 4),
 ('า', 4),
 ('ย', 4),
 ('3', 4),
 ('ไ', 4),
 ('ป', 4),
 ('ต', 4),
 ('า', 4),
 ('ม', 4),
 ('ถ', 4),
 ('.', 4),
 ('ป', 4),
 ('ิ', 4),
 ('่', 4),
 ('น', 4),
 ('เ', 4),
 ('ก', 4),
 ('ล', 4),
 ('้', 

In [26]:
demo = df[:][:40000]
demo

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [27]:
train = []
for i in range(len(demo)):
    train.append((demo[0][i], demo[1][i]))

In [ ]:
# train

In [28]:
import re
def remove_htmlTags(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
    
def isValidWord(text):
    dic = {text[0]:1}
    for c in text[1:]:
        if c not in dic:
            return True
        else:
            dic[c] += 1
    if dic[text[0]] > 1:
        return False
    else:
        return True
        
def tokenize(review):
    
#     tags = tltk.nlp.pos_tag(review.replace("\n", ""))
#     noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
#     cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
#     for i in range(len(cleaned)):
#         cleaned[i] = cleaned[i].replace("\n", "")
#         cleaned[i] = cleaned[i].replace("(", "")
#         cleaned[i] = cleaned[i].replace(")", "")
#     finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))

    # Filter parts of speech of text
    tags = tltk.nlp.pos_tag(remove_htmlTags(review.replace("\n", "")))
    finalWord = []
    filteredTags = {'NOUN', 'VERB', 'ADJ','ADV', 'AUX', 'PART'}
    for sentence in tags:
        for word, tag in sentence:
            if tag in filteredTags:
                if len(word) > 1 and isValidWord(word):
                    finalWord.append(remove_htmlTags(word))
    return finalWord

# tokenize(train[0][0])
# train[0][0]

In [ ]:
# from itertools import chain
# vocabulary = set(chain(*[tokenize(i[0].lower()) for i in train]))

In [ ]:
# vocabulary
tokenize(train[999][0])

In [29]:
from IPython.display import clear_output
corpus = []
for i in range(len(train)):
    print(str(i+1) + "/" + str(len(train)))
    corpus.append(tokenize(train[i][0]))

corpus
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

1/40000
2/40000


KeyboardInterrupt: 

In [30]:

import pickle


# pickle_out = open("corpus.pickle","wb")
# pickle.dump(corpus, pickle_out)
# pickle_out.close()


In [31]:
pickle_in = open("corpus3.pickle","rb")
corpus1 = pickle.load(pickle_in)
print(len(corpus1))

pickle_in = open("corpus_test.pickle","rb")
test_set = pickle.load(pickle_in)
len(test_set)
# corpus

40000


EOFError: Ran out of input

In [24]:
pickle_in = open("corpus2.pickle","rb")
corpus2 = pickle.load(pickle_in)
len(corpus2)

4124

In [54]:
# corpus
# corpus = corpus1 + corpus2

newcor = corpus1
test = test_set
# tltk.nlp.pos_tag(train[0][0])
len(newcor)

40000

In [55]:
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary = True)
tfidf_matrix = tfidf.fit_transform(newcor)

In [56]:
tfidf_matrix

<40000x171636 sparse matrix of type '<class 'numpy.float64'>'
	with 3587066 stored elements in Compressed Sparse Row format>

In [57]:

# idf = tfidf.idf_
# print (tfidf.get_feature_names(), idf)

In [58]:
# corpus = [
#     tokenize(df[0][0]),
#     tokenize(df[0][1]),
#     tokenize(df[0][2])
# ]

In [59]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# pca = PCA(n_components = 300)

# x_train = pca.fit_transform(x_train)
svd = TruncatedSVD(n_components=300, n_iter=10, random_state=42)
x_train = svd.fit_transform(tfidf_matrix)

y_train = []

for i in range(40000):
    y_train.append(train[i][1])
y_train = np.array(y_train)

In [60]:
print(x_train.shape)
# tfidf_matrix.toarray().shape
from sklearn.metrics import f1_score

(40000, 300)


In [61]:
from keras.utils import np_utils

tfidf_matrix = tfidf.transform(newcor[36000:])
# print(tfidf_matrix)
x_test = svd.transform(tfidf_matrix)

tfidf_matrix = tfidf.transform(test)
x_test_r = svd.transform(tfidf_matrix)
y_test = []


for i in range(36000,40000):
    y_test.append(train[i][1])
y_test = np.array(y_test)


# predictions = classifier.predict(x_test)
print(x_train.shape)
y_train.reshape(40000, 1)
y_train = np_utils.to_categorical(y_train)
y_train = y_train[:, 1:]
print(y_train)


(40000, 300)
[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]]


In [81]:
# x_train.mean()
# predictions
f1_score(y_test, predictions, average='macro')

ValueError: Found input variables with inconsistent numbers of samples: [1000, 5000]

In [138]:
predictions = linearClassifier.predict(x_test)
predictions
f1_score(y_test, predictions, average='macro')

/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.18891432308698497

In [137]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
from keras.utils import np_utils
from keras.layers import Conv1D, MaxPooling1D, Flatten

# from sklearn.decomposition import PCA

# pca = PCA(n_components = 300)
# x_train = tfidf_matrix.toarray()
# x_train = pca.fit_transform(x_train)
# y_train = []

# for i in range(3500):
#     y_train.append(train[i][1])
# y_train = np.array(y_train)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))
rate = 0.3

# def get_feedforward_nn():
#   input1 = Input(shape=(300,))
#   x = Dense(300, activation='relu')(input1)
#   x = Dropout(rate)(x)
#   x = Dense(128, activation='relu')(x)
#   x = Dropout(rate)(x)  
#   out = Dense(1, activation='relu')(x)

#   model = Model(inputs=input1, outputs=out)
#   model.compile(optimizer=Adam(),
#                 loss='mean_squared_error',
#                 metrics=[f1])
#   return model

import keras
from keras.models import Sequential
from keras.layers import Dense


# Initialising the ANN
classifier = Sequential()

#CNN
classifier.add(Conv1D(nb_filter = 128, filter_length = 3, input_shape = (300,1), padding = "same", activation = 'relu'))
classifier.add(MaxPooling1D())
# classifier.add(Dropout(0.3))

classifier.add(Conv1D(nb_filter = 128, filter_length = 3, padding = "same", activation = 'relu'))
classifier.add(MaxPooling1D())
# classifier.add(Dropout(0.2))
classifier.add(Flatten())

# Adding the input layer and the first hidden layer
# classifier.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(0.3))

# Adding the second hidden layer
# classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(0.3))

# Adding the output layer
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [f1])

# for c in y_train:
#     if c not in {1, 2, 3, 4, 5}:
#         print(y)
classifier.fit(np.expand_dims(x_train, axis = 2), y_train, batch_size = 256, epochs = 5)

Epoch 1/5
40000/40000 [==============================] - 72s - loss: 1.2214 - f1: nan    
Epoch 2/5
40000/40000 [==============================] - 73s - loss: 1.1722 - f1: nan    
Epoch 3/5
40000/40000 [==============================] - 73s - loss: 1.1361 - f1: 0.3093    
Epoch 4/5
40000/40000 [==============================] - 72s - loss: 1.1174 - f1: 0.3371    
Epoch 5/5
40000/40000 [==============================] - 73s - loss: 1.0971 - f1: 0.3590    


/home/ekapolc/.env/lib/python3.5/site-packages/ipykernel/__main__.py:77: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=128, activation="relu", input_shape=(300, 1), padding="same", kernel_size=3)`
/home/ekapolc/.env/lib/python3.5/site-packages/ipykernel/__main__.py:81: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(padding="same", filters=128, activation="relu", kernel_size=3)`


In [138]:
res = classifier.predict(np.expand_dims(x_test, axis = 2))
len(res)

4000

In [139]:
predictions = []
for i in range(len(res)):
    predictions.append(np.argmax(res[i]) + 1)
print(predictions)
from sklearn.metrics import f1_score,precision_score,recall_score
f1_score(y_test,predictions, average='micro')
# print(len(predictions))
# y_test.shape

[4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 3, 2, 4, 4, 3, 4, 4, 4, 4, 4, 3, 5, 4, 4, 2, 3, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 5, 4, 4, 4, 5, 3, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 3, 3, 3, 4, 5, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 3, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 2, 4, 5, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 5, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 2, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 5, 4, 

0.52649999999999997

In [131]:
df_res = pd.DataFrame({"reviewID": np.arange(1,len(res)+1),"rating": predictions}, columns=["reviewID", "rating"])
df_res.head(10)

,reviewID,rating
0,1,4
1,2,3
2,3,3
3,4,4
4,5,4
5,6,5
6,7,4
7,8,3
8,9,4
9,10,4


In [133]:
df_res.to_csv("result.csv", index=False)

In [451]:
from keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from keras import backend as K
K.clear_session()

# Path to save model parameters
weight_path_feedforward_nn='~/workspace/model/model_weight_feedforward_nn2.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.
callbacks_list_feedforward_nn = [
        TensorBoard(log_dir='~/workspace/model/Graph/ff2', histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

print(x_train.shape)
verbose = 1
model_feedforward_nn = get_feedforward_nn()
train_params = [(50, 64)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_feedforward_nn.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_test, y_test))

(3500, 300)
train with 50 epochs and 64 batch size


ValueError: Error when checking target: expected dense_3 to have shape (None, 1) but got array with shape (3500, 5)

In [65]:
# from keras.layers import Dropout

# def get_nn_with_dropout():
    
#     rate = 0.1
    
#     input1 = Input(shape=(21,))
#     x = Dense(100, activation='relu')(input1)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     out = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])
#     return model

from sklearn.metrics import f1_score,precision_score,recall_score

################################################################################
# Write a function to evaluate your model. Your function must make prediction  #
# using the input model and return f-score, precision, and recall of the model.#
# You can make predictions by calling model.predict().                         #
################################################################################
def evaluate(x_test, y_test, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)

#     #map probability to class
#     prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
#     y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
#     for i in range(len(y_pred)):
#         if y_pred[i] - int(y_pred[i]) >= 0.5:
#             y_pred[i] = int(y_pred[i]) + 1
#         else:
#             y_pred[i] = int(y_pred[i])
    print(y_pred.T[0])
    print(y_test)
    f1score = f1_score(y_test,y_pred.T[0], average='macro')
    precision = precision_score(y_test,y_pred,average='macro')
    recall = recall_score(y_test,y_pred,average='macro')
    return f1score, precision, recall

In [66]:
# print(x_test.shape)
# y_pred = classifier.predict(x_test)
evaluate(x_test, y_test, classifier)

ValueError: Error when checking : expected conv1d_15_input to have 3 dimensions, but got array with shape (1000, 300)